# Functions

Topics:
1. How to declare a function
2. Duck-typing in Julia
3. Mutating vs. non-mutating functions
4. Some higher order functions

## How to declare a function
Julia gives us a few different ways to write a function. The first requires the `function` and `end` keywords

In [1]:
function sayhi(name)
    println("Hi $name, it's great to see you!")
end

sayhi (generic function with 1 method)

In [2]:
function f(x)
    x^2
end

f (generic function with 1 method)

We can call either of these functions like this:

In [3]:
sayhi("C-3PO")

Hi C-3PO, it's great to see you!


In [4]:
f(42)

1764

Alternatively, we could have declared either of these functions in a single line

In [5]:
sayhi2(name) = println("Hi $name, it's great to see you!")

sayhi2 (generic function with 1 method)

In [6]:
f2(x) = x^2

f2 (generic function with 1 method)

In [7]:
sayhi2("R2D2")

Hi R2D2, it's great to see you!


In [8]:
f2(42)

1764

Finally, we could have declared these as "anonymous" functions

In [9]:
sayhi3 = name -> println("Hi $name, it's great to see you!")

(::#1) (generic function with 1 method)

In [10]:
f3 = x -> x^2

(::#3) (generic function with 1 method)

In [11]:
sayhi3("Chewbacca")

Hi Chewbacca, it's great to see you!


In [12]:
f3(42)

1764

## Duck-typing in Julia
*"If it quacks like a duck, it's a duck."* <br><br>
Julia functions will just work on whatever inputs make sense. <br><br>
For example, `sayhi` works on the name of this minor tv character, written as an integer...

In [13]:
sayhi(55595472)

Hi 55595472, it's great to see you!


And `f` will work on a matrix. 

In [15]:
A = rand(3, 3)
A

3×3 Array{Float64,2}:
 0.683498  0.627005   0.307168
 0.14587   0.0223539  0.567412
 0.192685  0.848259   0.101831

In [16]:
f(A)

3×3 Array{Float64,2}:
 0.617818  0.703131  0.596999
 0.212294  0.573273  0.115271
 0.275057  0.226156  0.550868

Unit test

In [19]:
f(A) == A^2  # as matrix product

true

`f` will also work on a string like "hi" because `*` is defined for string inputs as string concatenation.

In [20]:
f("hi")

"hihi"

On the other hand, `f` will not work on a vector. Unlike `A^2`, which is well-defined, the meaning of `v^2` for a vector, `v`, is ambiguous. 

In [21]:
v = rand(3)

3-element Array{Float64,1}:
 0.247577 
 0.582768 
 0.0295787

In [22]:
f(v)

LoadError: [91mDimensionMismatch("Cannot multiply two vectors")[39m

Remark: evaluate pointwisely with `.`

In [23]:
f.(v)

3-element Array{Float64,1}:
 0.0612944  
 0.339619   
 0.000874898

## Mutating vs. non-mutating functions

By convention, functions followed by `!` alter their contents and functions lacking `!` do not.

For example, let's look at the difference between `sort` and `sort!`.


In [26]:
v = [3, 5, 2]

3-element Array{Int64,1}:
 3
 5
 2

In [27]:
sort(v)

3-element Array{Int64,1}:
 2
 3
 5

In [28]:
v

3-element Array{Int64,1}:
 3
 5
 2

`sort(v)` returns a sorted array that contains the same elements as `v`, but `v` is left unchanged. <br><br>

On the other hand, when we run `sort!(v)`, the contents of v are sorted within the array `v`.

In [29]:
sort!(v)

3-element Array{Int64,1}:
 2
 3
 5

In [30]:
v

3-element Array{Int64,1}:
 2
 3
 5

## Some higher order functions

### map

`map` is a "higher-order" function in Julia that *takes a function* as one of its input arguments. 
`map` then applies that function to every element of the data structure you pass it. For example, executing

```julia
map(f, [1, 2, 3])
```
will give you an output array where the function `f` has been applied to all elements of `[1, 2, 3]`
```julia
[f(1), f(2), f(3)]
```

In [31]:
map(f, [1, 2, 3])

3-element Array{Int64,1}:
 1
 4
 9

Here we've squared all the elements of the vector `[1, 2, 3]`, rather than squaring the vector `[1, 2, 3]`.

To do this, we could have passed to `map` an anonymous function rather than a named function, such as

In [32]:
x -> x^3

(::#5) (generic function with 1 method)

via

In [33]:
map(x -> x^3, [1, 2, 3])

3-element Array{Int64,1}:
  1
  8
 27

and now we've cubed all the elements of `[1, 2, 3]`!

### broadcast

`broadcast` is another higher-order function like `map`. `broadcast` is a generalization of `map`, so it can do every thing `map` can do and more. The syntax for calling `broadcast` is the same as for calling `map`

In [25]:
?broadcast

search: broadcast broadcast! broadcast_getindex broadcast_setindex! bitbroadcast



```
broadcast(f, As...)
```

Broadcasts the arrays, tuples, `Ref`s, nullables, and/or scalars `As` to a container of the appropriate type and dimensions. In this context, anything that is not a subtype of `AbstractArray`, `Ref` (except for `Ptr`s), `Tuple`, or `Nullable` is considered a scalar. The resulting container is established by the following rules:

  * If all the arguments are scalars, it returns a scalar.
  * If the arguments are tuples and zero or more scalars, it returns a tuple.
  * If the arguments contain at least one array or `Ref`, it returns an array (expanding singleton dimensions), and treats `Ref`s as 0-dimensional arrays, and tuples as 1-dimensional arrays.

The following additional rule applies to `Nullable` arguments: If there is at least one `Nullable`, and all the arguments are scalars or `Nullable`, it returns a `Nullable` treating `Nullable`s as "containers".

A special syntax exists for broadcasting: `f.(args...)` is equivalent to `broadcast(f, args...)`, and nested `f.(g.(args...))` calls are fused into a single broadcast loop.

```jldoctest
julia> A = [1, 2, 3, 4, 5]
5-element Array{Int64,1}:
 1
 2
 3
 4
 5

julia> B = [1 2; 3 4; 5 6; 7 8; 9 10]
5×2 Array{Int64,2}:
 1   2
 3   4
 5   6
 7   8
 9  10

julia> broadcast(+, A, B)
5×2 Array{Int64,2}:
  2   3
  5   6
  8   9
 11  12
 14  15

julia> parse.(Int, ["1", "2"])
2-element Array{Int64,1}:
 1
 2

julia> abs.((1, -2))
(1, 2)

julia> broadcast(+, 1.0, (0, -2.0))
(1.0, -1.0)

julia> broadcast(+, 1.0, (0, -2.0), Ref(1))
2-element Array{Float64,1}:
 2.0
 0.0

julia> (+).([[0,2], [1,3]], Ref{Vector{Int}}([1,-1]))
2-element Array{Array{Int64,1},1}:
 [1, 1]
 [2, 2]

julia> string.(("one","two","three","four"), ": ", 1:4)
4-element Array{String,1}:
 "one: 1"
 "two: 2"
 "three: 3"
 "four: 4"

julia> Nullable("X") .* "Y"
Nullable{String}("XY")

julia> broadcast(/, 1.0, Nullable(2.0))
Nullable{Float64}(0.5)

julia> (1 + im) ./ Nullable{Int}()
Nullable{Complex{Float64}}()
```


In [34]:
broadcast(f, [1, 2, 3])

3-element Array{Int64,1}:
 1
 4
 9

and again, we've applied `f` (squared) to all the elements of `[1, 2, 3]` - this time by "broadcasting" `f`!

Some syntactic sugar for calling `broadcast` is to place a `.` between the name of the function you want to `broadcast` and its input arguments. For example,

```julia
broadcast(f, [1, 2, 3])
```
is the same as
```julia
f.([1, 2, 3])
```

In [35]:
f.([1, 2, 3])

3-element Array{Int64,1}:
 1
 4
 9

Notice again how different this is from calling 
```julia
f([1, 2, 3])
```
We can square every element of a vector, but we can't square a vector!

To drive home the point, let's look at the difference between

```julia
f(A)
```
and
```julia
f.(A)
```
for a matrix `A`:

In [36]:
A = [i + 3*j for j in 0:2, i in 1:3]

3×3 Array{Int64,2}:
 1  2  3
 4  5  6
 7  8  9

In [37]:
f(A)

3×3 Array{Int64,2}:
  30   36   42
  66   81   96
 102  126  150

As before we see that for a matrix, `A`,
```
f(A) = A^2 = A * A
``` 

On the other hand,

In [38]:
B = f.(A)

3×3 Array{Int64,2}:
  1   4   9
 16  25  36
 49  64  81

contains the squares of all the entries of `A`.

This dot syntax for broadcasting allows us to write relatively complex compound elementwise expressions in a way that looks natural/closer to mathematical notation. For example, we can write

In [39]:
A .+ 1 .* f.(A) ./ A

3×3 Array{Float64,2}:
  2.0   4.0   6.0
  8.0  10.0  12.0
 14.0  16.0  18.0

instead of

In [40]:
broadcast(x -> x + 1 * f(x) / x, A)

3×3 Array{Float64,2}:
  2.0   4.0   6.0
  8.0  10.0  12.0
 14.0  16.0  18.0

and this will still compile down to code that runs as efficiently as `C`!

### Exercises

#### 6.1 
Write a function that adds 1 to its input.

In [42]:
f4(x) = x + 1

(::#17) (generic function with 1 method)

#### 6.2 
Use `map` or `broadcast` to increment every element of matrix `A` by `1`.

In [43]:
broadcast(f4, A)

3×3 Array{Int64,2}:
 2  3   4
 5  6   7
 8  9  10

#### 6.3 
Use the broadcast dot syntax to increment every element of matrix `A` by `1`.

In [44]:
A .+ 1

3×3 Array{Int64,2}:
 2  3   4
 5  6   7
 8  9  10